# Assignment-I (Backpropagation Implementation)
## Deep Learning
## Wali Ullah (09745)

In [1]:
from sklearn.datasets import make_blobs
import numpy as np
import pandas as pd
import random 
from random import seed
from random import randrange
from random import random
from csv import reader
from math import exp
from sklearn.metrics import classification_report, confusion_matrix#for visualizing tree 
from sklearn.datasets import make_classification
from sklearn.preprocessing import MinMaxScaler

## data Generation and Transformation

In [2]:
def data(sample_size):
    X,y = make_classification(n_samples=sample_size, n_features=4, n_informative=4, 
                    n_redundant=0, n_repeated=0, n_classes=4, n_clusters_per_class=2,
                          class_sep=1.5,
                   flip_y=0,weights=[0.5,0.5,0.5,0.5])
    X = pd.DataFrame(X)
    y = pd.DataFrame(y)
    df= pd.concat([X, y], axis=1)
    #data=df
    df1= df.to_numpy()
    data=df1.astype(int)
    print('Sample data is generated')
    return data

def transformation(data):
    scaler = MinMaxScaler()
    scaler.fit(data)
    print('Min Max Transformation is applied to Sampled data')
    return data

# Alg with Neurons and Layers

In [3]:
# Alg with Neurons and Layers
#number of neurons=n_hidden
#number of layers=n_layers

In [4]:
from math import exp
from random import seed
from random import random

# Initialize a network with n_hidden neurons and n_layers number of layers


def initialize_network(n_inputs, n_hidden, n_layers, n_outputs):
    network = list()
    if n_layers==1:
            hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
            network.append(hidden_layer)
            output_layer = [{'weights':[random() for i in range(n_hidden)]} for i in range(n_outputs)]
            network.append(output_layer)
    else:
        for i in range(n_layers):
            if i==0:
                hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
                network.append(hidden_layer)
            else:
                hidden_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_hidden)]
                network.append(hidden_layer)
        output_layer = [{'weights':[random() for i in range(n_hidden)]} for i in range(n_outputs)]
        network.append(output_layer)
    return network

# Calculate neuron activation for an input
def activate(weights, inputs):
	activation = weights[-1]
	for i in range(len(weights)-1):
		activation += weights[i] * inputs[i]
	return activation



# Forward propagate input to a network output
def forward_propagate(network, row):
	inputs = row
	for layer in network:
		new_inputs = []
		for neuron in layer:
			activation = activate(neuron['weights'], inputs)
			neuron['output'] = transfer(activation, fn)
			new_inputs.append(neuron['output'])
		inputs = new_inputs
	return inputs



# Transfer neuron activation sigmoid
def transfer(activation, fn):
	z=fn
	if z=='sigmoid':
		t= 1.0 / (1.0 + exp(-activation))
	else:
		t=(exp(activation)-exp(-activation))/(exp(activation)+exp(-activation))
	return t



# Calculate the derivative of an neuron output sigmoind
def transfer_derivative(output, fn):
	z=fn
	if z=='sigmoid':
		t_deri=output * (1.0 - output)
	else:
		t_deri=1-(output)**2
	return t_deri


# Backpropagate error and store in neurons
def backward_propagate_error(network, expected,fn):
	for i in reversed(range(len(network))):
		layer = network[i]
		errors = list()
		if i != len(network)-1:
			for j in range(len(layer)):
				error = 0.0
				for neuron in network[i + 1]:
					error += (neuron['weights'][j] * neuron['delta'])
				errors.append(error)
		else:
			for j in range(len(layer)):
				neuron = layer[j]
				errors.append(expected[j] - neuron['output'])
		for j in range(len(layer)):
			neuron = layer[j]
			neuron['delta'] = errors[j] * transfer_derivative(neuron['output'], fn)

            
            
# Update network weights with error
def update_weights(network, row, l_rate):
	for i in range(len(network)):
		inputs = row[:-1]
		if i != 0:
			inputs = [neuron['output'] for neuron in network[i - 1]]
		for neuron in network[i]:
			for j in range(len(inputs)):
				neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
			neuron['weights'][-1] += l_rate * neuron['delta']

# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
	for epoch in range(n_epoch):
		sum_error = 0
		for row in train:
			outputs = forward_propagate(network, row)
			expected = [0 for i in range(n_outputs)]
			expected[row[-1]] = 1
			sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
			backward_propagate_error(network, expected, fn)
			update_weights(network, row, l_rate)
		print('&gt;epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

        
# Make a prediction with a network
def predict(network, row):
	outputs = forward_propagate(network, row)
	return outputs.index(max(outputs))

# Backpropagation Algorithm With Stochastic Gradient Descent
def back_propagation(train, test, l_rate,  n_epoch, n_hidden):
	n_inputs = len(train[0]) - 1
	n_outputs = len(set([row[-1] for row in train]))
	network = initialize_network(n_inputs, n_layers, n_hidden, n_outputs)
	train_network(network, train, l_rate, n_epoch, n_outputs)
	predictions = list()
	for row in test:
		prediction = predict(network, row)
		predictions.append(prediction)
	return(predictions)

# Split a dataset into a train and test set
def train_test_split(dataset, split=0.60):
	train = list()
	train_size = split * len(dataset)
	dataset_copy = list(dataset)
	while len(train) < train_size:
		index = randrange(len(dataset_copy))
		train.append(dataset_copy.pop(index))
	return train, dataset_copy
    
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
	#print("Classification report - \n", classification_report(actual, predicted))
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0    
    
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm,n_layers, *args):
	train, test = train_test_split(dataset)
    #folds = cross_validation_split(dataset, n_folds)
	scores = list()
	train_set = list(train)
	test_set = list()
	for row in train:
		row_copy = list(row)
		test_set.append(row_copy)
		row_copy[-1] = None
	predicted = algorithm(train_set, test_set, *args)
	actual = [row[-1] for row in test]
	accuracy = accuracy_metric(actual, predicted)
	scores.append(accuracy)
	return scores

    
    

In [5]:

    
# Test training backprop algorithm
seed(1)

sample_size=5000           #Choose sample size
data_raw= data(sample_size)
df1=transformation(data_raw)


dataset = df1
n_inputs = len(dataset[0]) - 1
n_outputs = len(set([row[-1] for row in dataset]))

#n_outputs=3

fn= 'sigmoid'      #choose function either 'sigmoid' or 'tanh'
l_rate = 0.9       #choose learning rate
n_epoch = 50      #chhose epoch size
n_hidden =5        #number of neurons in hidden layers
n_layers= 10       #number of hidden layers
scores = evaluate_algorithm(dataset,back_propagation, l_rate, n_layers, n_epoch, n_hidden)
print('Accuracy Scores: %s' % scores)


Sample data is generated
Min Max Transformation is applied to Sampled data
&gt;epoch=0, lrate=10.000, error=3008.055
&gt;epoch=1, lrate=10.000, error=2999.996
&gt;epoch=2, lrate=10.000, error=2999.996
&gt;epoch=3, lrate=10.000, error=2999.996
&gt;epoch=4, lrate=10.000, error=2999.995
&gt;epoch=5, lrate=10.000, error=2999.995
&gt;epoch=6, lrate=10.000, error=2999.994
&gt;epoch=7, lrate=10.000, error=2999.993
&gt;epoch=8, lrate=10.000, error=2999.992
&gt;epoch=9, lrate=10.000, error=2999.989
&gt;epoch=10, lrate=10.000, error=2999.979
&gt;epoch=11, lrate=10.000, error=3000.097
&gt;epoch=12, lrate=10.000, error=2999.996
&gt;epoch=13, lrate=10.000, error=2999.996
&gt;epoch=14, lrate=10.000, error=2999.996
&gt;epoch=15, lrate=10.000, error=2999.996
&gt;epoch=16, lrate=10.000, error=2999.995
&gt;epoch=17, lrate=10.000, error=2999.995
&gt;epoch=18, lrate=10.000, error=2999.994
&gt;epoch=19, lrate=10.000, error=2999.994
&gt;epoch=20, lrate=10.000, error=2999.993
&gt;epoch=21, lrate=10.000, erro